# Ratios Data: 
The ratios data can be fetched from the [NSE Website](https://www.niftyindices.com/reports/historical-data). 


In [1]:
import requests as rq
import pandas as pd
import numpy as np 
import json 
import time 
import datetime

In [2]:
# Home Page
url1 = "https://www.niftyindices.com/reports/historical-data"
 

In [3]:
# Create Session Object
session = rq.Session()

main_headers = {
    "Dnt": "1",
    "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
}

response1 = session.get(url1,headers=main_headers)

print(response1.status_code)



200


In [4]:

# Parse the Cookies from Home Page: 

home_page_cookies = response1.cookies.get_dict()
cookie_header = {"Cookie": "; ".join([f"{key}={value}" for key, value in home_page_cookies.items()])}

# Merge Cookies with Main Header: 

additional_header = { 'contentType': 'application/json; charset=utf-8', 
                      'dataType': 'json', 
                       'async': '!0'}

new_header_with_cookies =  main_headers | cookie_header | additional_header
print("Header Keys: ", new_header_with_cookies.keys())

Header Keys:  dict_keys(['Dnt', 'sec-ch-ua', 'sec-ch-ua-mobile', 'sec-ch-ua-platform', 'sec-fetch-dest', 'sec-fetch-mode', 'sec-fetch-site', 'sec-fetch-user', 'upgrade-insecure-requests', 'user-agent', 'Cookie', 'contentType', 'dataType', 'async'])


In [14]:
url2 = "https://www.niftyindices.com/Backpage.aspx/getHistoricaldatatabletoString"

payload =  {"cinfo":"{'name':'NIFTY 50','startDate':'28-Jun-2025','endDate':'01-Jul-2025','indexName':'NIFTY 50'}"}

In [15]:
# Finally, invoke API Request
response2 = session.post(url2,json=payload,headers=new_header_with_cookies)
print(response2.status_code)


200


In [16]:
json_data = json.loads(response2.text)

print(json_data)

{'d': '[{"RequestNumber":"His63887055124574239000","Index Name":"","INDEX_NAME":"Nifty 50","HistoricalDate":"01 Jul 2025","OPEN":"25551.35","HIGH":"25593.4","LOW":"25501.8","CLOSE":"25541.80"},{"RequestNumber":"His63887055124574239000","Index Name":"","INDEX_NAME":"Nifty 50","HistoricalDate":"30 Jun 2025","OPEN":"25661.65","HIGH":"25669.35","LOW":"25473.3","CLOSE":"25517.05"}]'}


In [20]:
# Step 1: Load the outer JSON
outer_json = json.loads(response2.text)

# Step 2: Parse the inner stringified JSON list
inner_data = json.loads(outer_json["d"])

# Step 3: Convert to DataFrame
df = pd.DataFrame(inner_data)

# Optional: Convert numeric columns to float
numeric_cols = ["OPEN", "HIGH", "LOW", "CLOSE"]
df[numeric_cols] = df[numeric_cols].astype(float)

# Optional: Convert HistoricalDate to datetime
df["HistoricalDate"] = pd.to_datetime(df["HistoricalDate"], format="%d %b %Y")

print(df)

             RequestNumber Index Name INDEX_NAME HistoricalDate      OPEN  \
0  His63887055124574239000              Nifty 50     2025-07-01  25551.35   
1  His63887055124574239000              Nifty 50     2025-06-30  25661.65   

       HIGH      LOW     CLOSE  
0  25593.40  25501.8  25541.80  
1  25669.35  25473.3  25517.05  


In [21]:
df

,RequestNumber,Index Name,INDEX_NAME,HistoricalDate,OPEN,HIGH,LOW,CLOSE
0,His63887055124574239000,,Nifty 50,2025-07-01,25551.35,25593.40,25501.8,25541.80
1,His63887055124574239000,,Nifty 50,2025-06-30,25661.65,25669.35,25473.3,25517.05
